Ноутбук для расписи всего алгоритма обработки данных

Импорт библиотек

In [1]:
import pandas as pd
import numpy as np
import re

# import seaborn as sns
# import matplotlib.pyplot as plt
# import missingno as msn

Загружаем данные

In [2]:
df = pd.read_csv('valid.csv')

C:\Users\egorb\AppData\Local\Temp\ipykernel_8000\407743444.py:1: DtypeWarning: Columns (51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,87,88,89,91,95,96,97,99,131,132,133,135,136,140,141,142,144,145,146,147,148,151,152,155,156,157,158,159,160,161,163,164,165,167,168,169,172,173,174,175,176,177,178,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,203,204,207,208,211,212,213,215,216,239,240,241,243,244,245,459,467,563,567,571,575,579,580,583,611,612,795,799,1031,1032,1033,1034,1035,1036,1037,1039,1040,1047,1055,1063,1064,1071,1072,1073,1074,1175,1176,1177,1178,1179,1180,1181,1182,1277,1278,1281,1282,1447,1448,1449,1450,1451,1452,1453,1454,1653,1654,1656,2193,2194,2195,2196,2199) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('valid.csv')


Убираем дупликаты и полностью пустые колонки

In [3]:
df.dropna(how='all', axis=1, inplace=True)
df.drop_duplicates()

,report_date,client_id,target,col2,col3,col4,col7,col8,col11,col12,...,col2654,col2655,col2656,col2657,col2658,col2659,col2660,col2661,col2662,col2663
0,2023-04-01,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4941670.0,4941670.0,4941670.0,NaN,NaN,NaN,4941670.0,4941670.0,4941670.0,0.256919
1,2023-05-01,4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,11789500.0,11789500.0,11789500.0,NaN,NaN,NaN,11789500.0,11789500.0,11789500.0,0.267221
2,2023-05-01,9,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7000000.0,7000000.0,7000000.0,NaN,NaN,NaN,7000000.0,7000000.0,7000000.0,0.257217
3,2023-03-01,12,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.256261
4,2023-03-01,14,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,10391000.0,10391000.0,10391000.0,0.257348
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3637,2023-04-01,4366,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3500000.0,3500000.0,3500000.0,0.256261
3638,2023-04-01,8298,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.256261
3639,2023-03-01,835,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.256919
3640,2023-03-01,8300,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.257217


Переводим все возможные колонки в INT64

In [4]:
float_columns = df.select_dtypes(include=np.number).columns
for col in float_columns:
    temp = df[col].fillna(0)
    if temp.apply(lambda x: x == int(x)).all():
        df[col] = df[col].astype("Int64")
df = df.loc[:, (df != 0).any(axis=0)]

Разбиваем датафрейм по типам данных

In [5]:
df_float = df.select_dtypes(include=["float64"])
df_int = df.select_dtypes(include=["int64"])
df_object = df.select_dtypes(include=["object"])

Merge'им каждый датафрейм

In [6]:
df_object = df_object.astype(str)
df_object.replace('0', '', inplace=True)
df_object.replace('nan', np.nan, inplace=True)

def merge_columns_with_common_values(df):
    global count
    count = 0
    columns = df.columns.tolist()
    merged_columns = []
    i = 0
    while i < len(columns) - 1:
        current_col = columns[i]
        next_col = columns[i + 1]
        
        common_values = list(set(df[current_col].unique()).intersection(set(df[next_col].unique())))
        x = np.nan
        if(x in common_values): common_values.remove(x)
        if common_values:
            count += 1
            merged_name = f"{current_col}"
            df[merged_name] = df[current_col].combine_first(df[next_col])
            merged_columns.append(merged_name)
            i += 2  
        else:
            merged_columns.append(current_col)
            i += 1
    
    if i == len(columns) - 1:
        merged_columns.append(columns[-1])
    
    df = df[merged_columns]
    return df

df_object = merge_columns_with_common_values(df_object)
i = 1
while count != 0:
    i += 1
    df_object = merge_columns_with_common_values(df_object)
    print(f"Iteration {i}: {len(df_object.columns)} columns left")

if count == 0:
    print("Nothing changed!")


Iteration 2: 86 columns left
Iteration 3: 45 columns left
Iteration 4: 40 columns left
Iteration 5: 40 columns left
Nothing changed!


In [7]:
def merge_columns_with_common_values(df):
    base_toggle = True
    count = 0
    unique_columns = []
    temp_col = "col4"
    
    df.replace(["nan", np.float64(0.0), np.float64(1.0)], np.nan, inplace=True)
    
    column_groups = {}

    for col in df.columns:
        if col in unique_columns:  continue
        
        if col != df.columns[1]: base_toggle = False

        column_groups[col] = [col]
        
        for next_col in df.columns:
            if col == next_col or next_col in unique_columns: continue
            
            common_values = list(set(df[col].unique()).intersection(set(df[next_col].unique())))
            if np.nan in common_values: common_values.remove(np.nan)
            
            match = re.match(r"([a-zA-Z]+)(\d{1,4})", col)
            match1 = re.match(r"([a-zA-Z]+)(\d{1,4})", next_col)
            def calc_jump():
                if base_toggle == True: return False
                else:
                    match = re.match(r"([a-zA-Z]+)(\d{1,4})", temp_col)
                    is_jump = ((int(match1.group(2)) - int(match.group(2))) > 5)
                    return is_jump

            if len(common_values) >= 1:
                if calc_jump():
                    temp_col = next_col
                    break
                count += 1
                temp_col = next_col
                column_groups[col].append(next_col)
                unique_columns.append(next_col)
                print(f"Merged {col} and {next_col}: common_values = {common_values}")
            else:
                if calc_jump():
                    temp_col = next_col
                    break
                common_values = list(set(df[temp_col].unique()).intersection(set(df[next_col].unique())))
                if len(common_values) >= 1:
                    count += 1
                    temp_col = next_col
                    column_groups[col].append(next_col) 
                    unique_columns.append(next_col) 
                    print(f"Merged {col} and {next_col}: common_values = {common_values}")
                else:break

        unique_columns.append(col)
    
    for group_name, group_columns in column_groups.items():
        if len(group_columns) > 1:
            df[group_name] = df[group_columns].bfill(axis=1).iloc[:, 0]
            print(f"Merged columns {group_columns} into {group_name}")
        else:
            df[group_name] = df[group_columns[0]]
    
    df = df[[col for col in df.columns if col in column_groups]]
    
    return df, count

df_float, count = merge_columns_with_common_values(df_float)

if count == 0:
    print("Nothing changed!")
else:
    print(f"{count} columns were merged.")



Merged col46 and col47: common_values = [np.float64(0.6)]
Merged col46 and col48: common_values = [np.float64(0.6)]
Merged col65 and col66: common_values = [np.float64(0.1), np.float64(0.3333333333333333), np.float64(0.0909090909090909)]
Merged col65 and col67: common_values = [np.float64(0.1), np.float64(0.3333333333333333), np.float64(0.0909090909090909)]
Merged col65 and col68: common_values = [np.float64(0.1), np.float64(0.3333333333333333), np.float64(0.0909090909090909)]
Merged col65 and col69: common_values = [np.float64(0.5)]
Merged col65 and col70: common_values = [np.float64(0.5)]
Merged col65 and col71: common_values = [np.float64(0.1), np.float64(0.3333333333333333)]
Merged col65 and col72: common_values = [np.float64(0.1), np.float64(0.3333333333333333), np.float64(0.0909090909090909)]
Merged col65 and col73: common_values = [np.float64(0.3333333333333333), np.float64(0.08), np.float64(0.0909090909090909)]
Merged col65 and col74: common_values = [np.float64(0.3333333333333

In [8]:
df_int = df_int.astype(float)

df_int = df_int.drop(["client_id","target"],axis=1)
df_int.replace("nan",np.nan, inplace=True)

def merge_columns_with_common_values(df):
    base_toggle = True
    count = 0
    unique_columns = []
    temp_col = "col2"

    column_groups = {}

    for col in df.columns:
        if col in unique_columns: continue

        if col != df.columns[1]: base_toggle = False

        column_groups[col] = [col]

        for next_col in df.columns:
            if col == next_col or next_col in unique_columns:
                continue

            common_values = list(set(df[col].unique()).intersection(set(df[next_col].unique())))
            if np.nan in common_values:
                common_values.remove(np.nan)

            match = re.match(r"([a-zA-Z]+)(\d{1,4})", col)
            match1 = re.match(r"([a-zA-Z]+)(\d{1,4})", next_col)
            def calc_jump():
                if base_toggle:return False
                else:
                    match = re.match(r"([a-zA-Z]+)(\d{1,4})", temp_col)
                    is_jump = (int(match1.group(2)) - int(match.group(2))) > 5
                    return is_jump

            if (len(common_values) >= 1):
                if calc_jump():
                    temp_col = next_col
                    break
                count += 1
                temp_col = next_col
                column_groups[col].append(next_col)
                unique_columns.append(next_col)
                print(f"Merged {col} and {next_col}: common_values = {common_values}")
            else:
                if calc_jump():
                    temp_col = next_col
                    break
                common_values = list(
                    set(df[temp_col].unique()).intersection(set(df[next_col].unique()))
                )
                if len(common_values) >= 1:
                    count += 1
                    temp_col = next_col
                    column_groups[col].append(next_col)
                    unique_columns.append(next_col)
                    print(f"Merged {col} and {next_col}: common_values = {common_values}")
                else:break

        unique_columns.append(col)

    for group_name, group_columns in column_groups.items():
        if len(group_columns) > 1:
            df[group_name] = (df[group_columns].bfill(axis=1).iloc[:, 0])
            print(f"Merged columns {group_columns} into {group_name}")
        else:
            df[group_name] = df[group_columns[0]]

    df = df[[col for col in df.columns if col in column_groups]]

    return df, count

df_int, count = merge_columns_with_common_values(df_int)

if count == 0: print("Nothing changed!")
else: print(f"{count} columns were merged.")

Merged col8 and col11: common_values = [np.float64(3353520.0)]
Merged col8 and col12: common_values = [np.float64(3353520.0)]
Merged col15 and col16: common_values = [np.float64(3465580.0)]
Merged col15 and col19: common_values = [np.float64(3353520.0)]
Merged col15 and col20: common_values = [np.float64(3353520.0)]
Merged col15 and col23: common_values = [np.float64(3353520.0)]
Merged col15 and col24: common_values = [np.float64(3353520.0)]
Merged col27 and col28: common_values = [np.float64(20925572.0)]
Merged col31 and col32: common_values = [np.float64(7868070.0)]
Merged col39 and col40: common_values = [np.float64(7868070.0)]
Merged col39 and col43: common_values = [np.float64(7868070.0)]
Merged col39 and col44: common_values = [np.float64(7868070.0)]
Merged col101 and col102: common_values = [np.float64(-8.0), np.float64(-13.0)]
Merged col101 and col103: common_values = [np.float64(-8.0)]
Merged col101 and col104: common_values = [np.float64(-8.0)]
Merged col101 and col105: commo

Соединяем обработаные фреймы в один

In [9]:
df_not_sorted = pd.concat([df_object['report_date'],df['client_id'], df['target']],axis=1)
df_object.drop(columns='report_date', inplace=True)

df_not_sorted = pd.concat([df_not_sorted, df_object, df_int, df_float],axis=1)
df_not_sorted.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3642 entries, 0 to 3641
Columns: 206 entries, report_date to col2663
dtypes: Int64(2), float64(164), object(40)
memory usage: 5.7+ MB


Сортируем колонки во фрейме

In [10]:
order_dict = {}
order_list = ['report_date','client_id','target']
columns = df_not_sorted.columns
for column in columns:
    if 'col' in column:
        order_dict[column] = int(re.match(r"([a-zA-Z]+)(\d{1,4})", column).group(2))

sorted_columns = {k: v for k, v in sorted(order_dict.items(), key=lambda item: item[1])}
for k in sorted_columns:
    order_list.append(k)

df = df_not_sorted[order_list]

Удаляем столбцы со ссылками

In [11]:
def has_link(column): return column.astype(str).str.startswith(('http://', 'https://'), na=False).any()

df = df.loc[:, ~df.apply(has_link)]

На этом наша обработка закончена, сохраняем файл

In [12]:
df.to_csv("processed_valid.csv", index=False)